# Install MuJoCo, MJX, and Brax

In [ ]:
!pip install -q mujoco wandb mujoco_mjx brax ml_collections
#!pip install -q --upgrade ipykernel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 115.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 114.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 998.9/998.9 kB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.1/309.1 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.9 MB/s eta 0:00:00
   ━

In [1]:
#@title Check if MuJoCo installation was successful

import distutils.util
import os
import subprocess
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')

# Add an ICD config so that glvnd can pick up the Nvidia EGL driver.
# This is usually installed as part of an Nvidia driver package, but the Colab
# kernel doesn't install its driver via APT, and as a result the ICD is missing.
# (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

try:
  print('Checking that the installation succeeded:')
  import mujoco
  mujoco.MjModel.from_xml_string('<mujoco/>')
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')

print('Installation successful.')

# Tell XLA to use Triton GEMM, this improves steps/sec by ~30% on some GPUs
xla_flags = os.environ.get('XLA_FLAGS', '')
xla_flags += ' --xla_gpu_triton_gemm_any=True'
os.environ['XLA_FLAGS'] = xla_flags


/tmp/ipykernel_43383/1450797713.py:3: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils.util


Sun Aug 18 14:00:24 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.06              Driver Version: 555.42.06      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1660 Ti     Off |   00000000:01:00.0  On |                  N/A |
| N/A   45C    P3             27W /   80W |    1129MiB /   6144MiB |     10%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
#@title Import packages for plotting and creating graphics
import time
import itertools
import numpy as np
from typing import Callable, NamedTuple, Optional, Union, List

# Graphics and plotting.
print('Installing mediapy:')
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy
import mediapy as media
import matplotlib.pyplot as plt

# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)

Installing mediapy:


In [3]:
#@title Import MuJoCo, MJX, and Brax
from datetime import datetime
from etils import epath
import functools
from IPython.display import HTML
from typing import Any, Dict, Sequence, Tuple, Union
import os
from ml_collections import config_dict


import jax
from jax import numpy as jp
import numpy as np
from flax.training import orbax_utils
from flax import struct
from matplotlib import pyplot as plt
import mediapy as media
from orbax import checkpoint as ocp

import mujoco
from mujoco import mjx

from brax import base
from brax import envs
from brax import math
from brax.base import Base, Motion, Transform
from brax.envs.base import Env, PipelineEnv, State
from brax.mjx.base import State as MjxState
from brax.training.agents.ppo import train as ppo
from brax.training.agents.ppo import networks as ppo_networks
from brax.io import html, mjcf, model

import wandb

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.2) or chardet (5.2.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


#login to wandb

In [ ]:
# api_token = '50c8814867c630e4f1649261257a9e728b7cebb6'
# try:
#     if wandb.login(key=api_token, relogin=False):
#         print("Successfully logged in to Weights & Biases.")
#     else:
#         print("Login failed. Please check your API token.")
# except Exception as e:
#     print(f"An error occurred: {e}")

# Experiment Config

In [4]:
config={
    "algorithm": "PPO",
    "Policy Network Architecture": "default",
    "Value Network Architecture": "default",
    "Obs":"data.qpos[2:]+data.qvel[3:6]+data.qacc[0:2]",
    "reward":"multi-stage",
    "num_timesteps": 30_000_000,
    "num_evals":5,
    "reward_scaling":0.1,
    "episode_length":1000,
    "normalize_observations": True,
    "action_repeat": 1,
    "unroll_length": 10,
    "num_minibatches": 32,
    "num_updates_per_batch": 8,
    "discounting": 0.97,
    "learning_rate": 3e-4,
    "entropy_cost": 1e-3,
    "num_envs": 2048,
    "batch_size": 1024,
    "seed": 0,
    "xml_path": None,
    "restore_checkpoint_path": None,
    "create_checkpoint_path": None
}

# Utilities

In [5]:
# a utility function to create a subset from a given config
sub_config = lambda config, wanted_keys: dict((k, config[k]) for k in wanted_keys if k in config)

# Train

In [ ]:
#@title Humanoid Env

class Humanoid(PipelineEnv):

  def __init__(
      self,
      episode_length,
      reset_noise_scale=1e-2,
      **kwargs,
  ):
    path = epath.Path(epath.resource_path('mujoco')) / (
        'mjx/test_data/humanoid'
    )
    mj_model = mujoco.MjModel.from_xml_path(
        (path / 'humanoid.xml').as_posix())
    mj_model.opt.solver = mujoco.mjtSolver.mjSOL_CG
    mj_model.opt.iterations = 6
    mj_model.opt.ls_iterations = 6

    sys = mjcf.load_model(mj_model)

    physics_steps_per_control_step = 5
    kwargs['n_frames'] = kwargs.get(
        'n_frames', physics_steps_per_control_step)
    kwargs['backend'] = 'mjx'

    super().__init__(sys, **kwargs)




    Cxy = jp.array([1, 0])
    self._xy_velocity = lambda x: 0.15*jp.exp(-5*jp.square(x[:2]-Cxy))

    qd = lambda x,y: jp.linalg.norm(x-y)

    C_orient = jp.array([1, 0, 0, 0]) 
    self._orient = lambda x: 0.1*jp.exp(-30*qd(x,C_orient))
    self._contract
















    def gaussian_factory(mean, std, magnitude=1, offset=0):
      return lambda x: magnitude*jp.exp(
          -0.5*jp.power(
              (x-mean)/std
              , 2
            )
          )+offset

    def bounded_exponential_factory(x, x_max, k=1.0):
      return lambda x: 100 * (1 - jp.exp(-k * x)) / (1 - jp.exp(-k * x_max))

    self._y_deviation_penalty = gaussian_factory(0, 2, 1, -1)

    target_velocity = 1
    x_max = 1e3
    self._x_position_reward = lambda x: jp.clip(10*(1-jp.exp(-(x/x_max))), 0, 10)

    self._x_velocity_reward = gaussian_factory(target_velocity, target_velocity, 2, 0)

    self._control_cost = lambda x:  -jp.sum(jp.square(x[:-6]))/30000-jp.sum(jp.square(x[-6:]))

    self._healthy_reward = lambda x: jp.where(x < 1.0, 0.0, 1.0)

    self._default_pose = jp.array([0, 0, 1.282, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])#sys.mj_model.keyframe('home').qpos[7:]

    self._stand_still_penalty = lambda x: -jp.sum(jp.abs(x - self._default_pose))/42

    self._reset_noise_scale = reset_noise_scale

  def reset(self, rng: jp.ndarray) -> State:
    """Resets the environment to an initial state."""
    rng, rng1, rng2 = jax.random.split(rng, 3)

    low, hi = -self._reset_noise_scale, self._reset_noise_scale
    qpos = self.sys.qpos0 + jax.random.uniform(
        rng1, (self.sys.nq,), minval=low, maxval=hi
    )
    qvel = jax.random.uniform(
        rng2, (self.sys.nv,), minval=low, maxval=hi
    )


    data = self.pipeline_init(qpos, qvel)

    reward, done, zero = jp.zeros(3)
    metrics = {
      'y_deviation_penalty': zero,
      'x_position_reward': zero,
      #'x_velocity_reward': zero,
      'control_cost': zero,
      'healthy_reward': zero,
      'x_position': zero,
      'y_position': zero,
      'x_velocity': zero,
      'y_velocity': zero,
    }

    info = {'counter':zero}

    obs = self._get_obs(data, jp.zeros(self.sys.nu), info['counter'])

    return State(data, obs, reward, done, metrics, info)

  def step(self, state: State, action: jp.ndarray) -> State:
    """Runs one timestep of the environment's dynamics."""
    action = jp.where(
        state.info['counter'] % 200 <= 99,
        action,
        jp.concatenate([
          action[0:3],
          action[9:15],
          action[3:9],
          action[18:21],
          action[15:18]
        ])
    )
    data0 = state.pipeline_state
    data = self.pipeline_step(data0, action)

    com_before = data0.subtree_com[1]
    com_after = data.subtree_com[1]
    velocity = (com_after - com_before) / self.dt

    metrics = {
      'xy_velocity': self._xy_velocity(velocity),


      'y_deviation_penalty': self._y_deviation_penalty(data.q[1]),
      'x_position_reward': self._x_position_reward(data.q[0]),
      #'x_velocity_reward': self._x_velocity_reward(velocity[0]),
      'control_cost': self._control_cost(action),
      'healthy_reward': self._healthy_reward(data.q[2]),
    }

    state.info['counter'] += 1

    reward = jp.array(sum(metrics.values()))

    done = jp.where(data.q[2] < 0.9, 1.0, 0.0)

    obs = self._get_obs(data, action, state.info['counter'])

    state.metrics.update(
        x_position=com_after[0],
        y_position=com_after[1],
        x_velocity=velocity[0],
        y_velocity=velocity[1],
        **metrics
    )
    return state.replace(
        pipeline_state=data, obs=obs, reward=reward, done=done, info = state.info
    )

  def _get_obs(
      self, data: mjx.Data, action: jp.ndarray, counter: jp.ndarray
  ) -> jp.ndarray:
    """Observes humanoid body position, velocities, and angles."""
    # external_contact_forces are excluded
    return jp.where(
        counter % 200 <= 99,
        jp.concatenate([
          data.qpos[2:],
          data.qvel[3:],
          action
        ]),
        jp.concatenate([
          data.qpos[2:10],
          data.qpos[16:22],
          data.qpos[10:16],
          data.qpos[25:28],
          data.qpos[22:25],
          data.qvel[3:9],
          data.qvel[15:21],
          data.qvel[9:15],
          data.qvel[24:27],
          data.qvel[21:24],
          action[0:3],
          action[9:15],
          action[3:9],
          action[18:21],
          action[15:18]
        ])
    )



envs.register_environment('humanoid', Humanoid)

In [ ]:

env_name = 'humanoid'
env_config = sub_config(config, ['episode_length'])
env = envs.get_environment(env_name, **env_config)

In [ ]:
wandb.init(
    project="tmp",
    config=config
)

## Train Humanoid Policy

In [ ]:
ckpt_path = epath.Path('/content/tmp/humanoid/ckpts')
ckpt_path.mkdir(parents=True, exist_ok=True)

def policy_params_fn(current_step, make_policy, params):
  # save checkpoints
  # orbax_checkpointer = ocp.PyTreeCheckpointer()
  # save_args = orbax_utils.save_args_from_target(params)
  # path = ckpt_path / f'{current_step}'
  # orbax_checkpointer.save(path, params, force=True, save_args=save_args)
  pass

def save_checkpoint(path, params):
  orbax_checkpointer = ocp.PyTreeCheckpointer()
  save_args = orbax_utils.save_args_from_target(params)
  orbax_checkpointer.save(path, params, force=True, save_args=save_args)

train_config = sub_config(config, ['num_timesteps', 'num_evals', 'reward_scaling', 'episode_length', 'normalize_observations', 'action_repeat', 'unroll_length', 'num_minibatches', 'num_updates_per_batch', 'discounting', 'learning_rate', 'entropy_cost', 'num_envs', 'batch_size', 'seed'])

# make_networks_factory = functools.partial(
#     ppo_networks.make_ppo_networks,
#         policy_hidden_layer_sizes=(16, 32, 32)
# )

train_fn = functools.partial(ppo.train, **train_config)


times = [datetime.now()]
def progress(num_steps, metrics):
  times.append(datetime.now())
  _metrics = metrics.copy()
  _metrics['numsteps'] = num_steps
  wandb.log(_metrics)



make_inference_fn, params, _= train_fn(environment=env, progress_fn=progress)

print(f'time to jit: {times[1] - times[0]}')
print(f'time to train: {times[-1] - times[1]}')

In [ ]:
save_checkpoint(ckpt_path, params)

## Visualize Policy

In [ ]:
eval_env = envs.get_environment(env_name,episode_length=1000)

jit_reset = jax.jit(eval_env.reset)
jit_step = jax.jit(eval_env.step)
inference_fn = make_inference_fn(params)
jit_inference_fn = jax.jit(inference_fn)

In [ ]:
!mkdir video

In [ ]:
# initialize the state
rng = jax.random.PRNGKey(0)
state = jit_reset(rng)
rollout = [state.pipeline_state]

# grab a trajectory
n_steps = 1000
render_every = 2

for i in range(n_steps):
  act_rng, rng = jax.random.split(rng)
  ctrl, _ = jit_inference_fn(state.obs, act_rng)
  state = jit_step(state, ctrl)
  rollout.append(state.pipeline_state)

  if state.done:
    break

media.write_video('video/eval.mp4',env.render(rollout[::render_every], camera='side'), fps=1.0 / env.dt / render_every)
#media.show_video(env.render(rollout[::render_every], camera='side'), fps=1.0 / env.dt / render_every)

In [ ]:
wandb.log({"video": wandb.Video("video/eval.mp4", caption="tmp" ,fps=1.0 / env.dt / render_every, format="mp4")})

In [ ]:
import zipfile
def zip_directory(folder_path, zip_path):
    # Ensure the folder exists
    if not os.path.exists(folder_path):
        print(f"The directory {folder_path} does not exist.")
        return

    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file),
                           os.path.relpath(os.path.join(root, file),
                                           os.path.join(folder_path, '..')))
    print(f"Successfully created zip file at {zip_path}")

# Define the folder and zip file paths
folder_path = "/content/tmp/humanoid/ckpts"
zip_path = "/content/ckpts.zip"

# Call the function to zip the directory
zip_directory(folder_path, zip_path)

In [ ]:
wandb.save(zip_path, base_path="/content/")

In [ ]:
%notebook -e notebook.ipynb

In [ ]:
wandb.save("/content/notebook.ipynb", base_path="/content/")

In [ ]:
wandb.finish()